# Import the required libraries and modules that you would need.

In [234]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Read that data into Python and call the dataframe donors.

In [235]:
target = pd.read_csv('target7_02.csv')
target = target.drop(['Unnamed: 0'], axis=1)

In [236]:
categorical = pd.read_csv('categorical7_02.csv')
categorical = categorical.drop(['Unnamed: 0'], axis=1)

In [237]:
numerical = pd.read_csv('numerical7_02.csv')
numerical = numerical.drop(['Unnamed: 0'], axis=1)

In [238]:
donors = pd.concat([numerical, categorical, target], axis = 1)
donors

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,SOLIH,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,TARGET_B,TARGET_D
0,8901,0,3712,60.000000,5.0,6.0,0,0,39,34,18,10,2,1,5.0,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,9702,74,6,14,240.0,31,14,5.0,9208,12.0,9402,10.0,9512,8911,9003.0,4.0,7.741935,95515,0,4,39.0,IL,36,U,F,3,NaN,NaN,L,E,C,T,2,0,0.0
1,9401,1,5202,46.000000,6.0,9.0,16,0,15,55,11,6,2,1,9.0,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,9702,32,6,13,47.0,3,1,10.0,9310,25.0,9512,25.0,9512,9310,9504.0,18.0,15.666667,148535,0,2,1.0,CA,14,H,M,3,NaN,NaN,L,G,A,S,1,0,0.0
2,9001,1,0,61.611649,3.0,1.0,2,0,20,29,33,6,8,1,1.0,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,

# Check the datatypes of all the columns in the data.

In [239]:
donors.dtypes

ODATEDW       int64
TCODE         int64
DOB           int64
AGE         float64
INCOME      float64
             ...   
GEOCODE2     object
DOMAIN_A     object
DOMAIN_B      int64
TARGET_B      int64
TARGET_D    float64
Length: 337, dtype: object

# Check for null values in the dataframe. Replace the null values using the methods learned in class.

In [240]:
nulls_percent_df = pd.DataFrame(donors.isna().sum()/len(donors)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

,column_name,nulls_percentage
0,ODATEDW,0.0
1,TCODE,0.0
2,DOB,0.0
3,AGE,0.0
4,INCOME,0.0
...,...,...
332,GEOCODE2,0.0
333,DOMAIN_A,0.0
334,DOMAIN_B,0.0
335,TARGET_B,0.0


In [241]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

,column_name,nulls_percentage
316,NEXTDATE,0.104526
328,SOLIH,0.935019
329,VETERANS,0.890727


In [242]:
donors['NEXTDATE'].describe().T
# the range in this column is not too big so I will fill the NaN's with the mean

count    85439.000000
mean      9151.022917
std        294.257260
min       7211.000000
25%       8903.000000
50%       9204.000000
75%       9409.000000
max       9702.000000
Name: NEXTDATE, dtype: float64

In [243]:
donors['NEXTDATE'] = donors['NEXTDATE'].fillna(np.mean(donors['NEXTDATE']))
donors['NEXTDATE'].isna().sum().sum()

0

In [244]:
donors['SOLIH'].describe().T

count    6200.000000
mean       11.084355
std         3.088073
min         0.000000
25%        12.000000
50%        12.000000
75%        12.000000
max        12.000000
Name: SOLIH, dtype: float64

In [245]:
donors['SOLIH'].value_counts(dropna=False)
# checking the documentation it shows there is also a category 5. So I will repalce the NaN's with 5

NaN     89212
12.0     5693
0.0       296
1.0        94
2.0        75
3.0        19
4.0        16
6.0         7
Name: SOLIH, dtype: int64

In [246]:
donors['SOLIH'] = donors['SOLIH'].fillna(5.0)
donors['SOLIH'].isna().sum().sum()

0

In [247]:
donors['VETERANS'].value_counts(dropna=False)
# I will replace NaN's with no as this is the only other option

NaN    84986
Y      10426
Name: VETERANS, dtype: int64

In [248]:
donors['VETERANS'] = donors['VETERANS'].fillna('N')
donors['VETERANS'].isna().sum().sum()

0

# Split the data into numerical and catagorical. Decide if any columns need their dtype changed.

In [249]:
y = donors['TARGET_B']
X = donors.drop(['TARGET_B','TARGET_D'], axis=1)

In [250]:
X_num = X.select_dtypes(np.number)
X_cat = X.select_dtypes(np.object)

C:\Users\jasmi\AppData\Local\Temp\ipykernel_17868\7884191.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_cat = X.select_dtypes(np.object)


In [251]:
X_cat

,STATE,HOMEOWNR,GENDER,VETERANS,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
0,IL,U,F,N,L,E,C,T
1,CA,H,M,N,L,G,A,S
2,NC,U,M,N,L,E,C,R
3,CA,U,F,N,L,E,C,R
4,FL,H,F,N,L,F,A,S
...,...,...,...,...,...,...,...,...
95407,other,U,M,N,L,G,C,C
95408,TX,H,M,N,L,F,A,C
95409,MI,U,M,N,L,E,B,C
95410,CA,H,F,N,L,F,A,C


In [252]:
X_num['INCOME'].value_counts(dropna=False)
# I will change this column to categorical. I will use X for that as I will use X later for the x y split

5.0    36737
2.0    13114
4.0    12732
1.0     9022
3.0     8558
6.0     7778
7.0     7471
Name: INCOME, dtype: int64

In [253]:
X['INCOME'] = X['INCOME'].astype('object')

## Split the data into a training set and a test set.

In [254]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_test_df  = pd.DataFrame(X_test,  columns=X.columns)

X_train_num = X_train_df.select_dtypes(include = np.number)
X_test_num  = X_test_df.select_dtypes(include = np.number)
X_train_cat = X_train_df.select_dtypes(include = np.object)
X_test_cat  = X_test_df.select_dtypes(include = np.object)

C:\Users\jasmi\AppData\Local\Temp\ipykernel_17868\3882879824.py:8: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_cat = X_train_df.select_dtypes(include = np.object)
C:\Users\jasmi\AppData\Local\Temp\ipykernel_17868\3882879824.py:9: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_cat  = X_test_df.select_dtypes(include = np.object)


## Scale the features either by using normalizer or a standard scaler.

In [255]:
transformer = MinMaxScaler().fit(X_train_num) 
X_train_scaled_arr = transformer.transform(X_train_num)
X_train_scaled = pd.DataFrame(X_train_scaled_arr, columns=X_train_num.columns)

In [256]:
X_test_scaled_arr = transformer.transform(X_test_num)
X_test_scaled = pd.DataFrame(X_test_scaled_arr, columns=X_test_num.columns)

## Encode the categorical features using One-Hot Encoding or Ordinal Encoding

In [257]:
encoder = OneHotEncoder(drop='first').fit(X_train_cat)
encoded_cat = encoder.transform(X_train_cat).toarray()
cols = encoder.get_feature_names(input_features=X_train_cat.columns)
onehot_encoded_train = pd.DataFrame(encoded_cat, columns=cols)

C:\Users\jasmi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [258]:
encoded_test_cat = encoder.transform(X_test_cat).toarray()
onehot_encoded_test = pd.DataFrame(encoded_test_cat, columns=cols)

## Fit a logistic regression model on the training data.

In [279]:
X_test_treated = pd.concat([X_test_scaled, onehot_encoded_test], axis=1)
X_train_treated = pd.concat([X_train_scaled, onehot_encoded_train], axis=1)

In [260]:
classification = LogisticRegression(solver='lbfgs') 
classification.fit(X_train_scaled, y_train)

C:\Users\jasmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

## Check the accuracy on the test data

In [261]:
predictions = classification.predict(X_test_scaled)
confusion_matrix(y_test, predictions)

array([[18080,     0],
       [ 1003,     0]], dtype=int64)

In [262]:
pred = classification.predict(X_test_scaled)
print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

# I dont understand why precision here is 0. 

precision:  0.0
recall:  0.0
f1:  0.0


C:\Users\jasmi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Check for the imbalance.

In [263]:
y.value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

# Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.

In [264]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True) 

train = pd.concat([X_train_scaled, y_train],axis=1)

## Oversampling

In [265]:
no_donor_over = train[train['TARGET_B']==0]
yes_donor_over = train[train['TARGET_B']==1]

yes_donor_oversampled = resample(yes_donor_over, 
                                    replace=True, 
                                    n_samples = len(no_donor_over))

train_oversampled = pd.concat([no_donor_over,yes_donor_oversampled],axis=0)

y_train_over = train_oversampled['TARGET_B'].copy()
X_train_over = train_oversampled.drop('TARGET_B',axis = 1).copy()

In [266]:
y_train_over.value_counts()

0    72489
1    72489
Name: TARGET_B, dtype: int64

## Undersampling

In [267]:
no_donor_under = train[train['TARGET_B']==0]
yes_donor_under = train[train['TARGET_B']==1]

no_donor_undersampled = resample(no_donor_under, 
                                    replace=False, 
                                    n_samples = len(yes_donor_under))

train_undersampled = pd.concat([yes_donor_under,yes_donor_undersampled],axis=0)

y_train_under = train_undersampled['TARGET_B'].copy()
X_train_under = train_undersampled.drop('TARGET_B',axis = 1).copy()

In [268]:
y_train_under.value_counts()

0    3921
1    3840
Name: TARGET_B, dtype: int64

## SMOTE

In [269]:
sm = SMOTE(random_state=100, k_neighbors=4)
X_train_SMOTE,y_train_SMOTE = sm.fit_resample(X_train_scaled,y_train)

# Each time fit the model and see how the accuracy of the model has changed.

## Oversampling

In [270]:
LR_over = LogisticRegression(solver='lbfgs') 
LR_over.fit(X_train_over, y_train_over)

C:\Users\jasmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [271]:
predictions_over = LR_over.predict(X_test_scaled)
confusion_matrix(y_test, predictions_over)

array([[11239,  6841],
       [  486,   517]], dtype=int64)

In [272]:
pred_over = LR_over.predict(X_test_scaled)
print("precision: ",precision_score(y_test,pred_over))
print("recall: ",recall_score(y_test,pred_over))
print("f1: ",f1_score(y_test,pred_over))

precision:  0.07026365860288122
recall:  0.5154536390827518
f1:  0.12366941753378785


## Undersampling

In [273]:
LR_under = LogisticRegression(solver='lbfgs') 
LR_under.fit(X_train_under, y_train_under)

C:\Users\jasmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [274]:
predictions_under = LR_under.predict(X_test_scaled)
confusion_matrix(y_test, predictions_under)

array([[  911, 17169],
       [   33,   970]], dtype=int64)

In [275]:
pred_under = LR_under.predict(X_test_scaled)
print("precision: ",precision_score(y_test,pred_under))
print("recall: ",recall_score(y_test,pred_under))
print("f1: ",f1_score(y_test,pred_under))

precision:  0.053475935828877004
recall:  0.967098703888335
f1:  0.10134782154424825


## SMOTE

In [276]:
LR_smote = LogisticRegression(solver='lbfgs') 
LR_smote.fit(X_train_SMOTE, y_train_SMOTE)

C:\Users\jasmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [277]:
predictions_smote = LR_smote.predict(X_test_scaled)
confusion_matrix(y_test, predictions_smote)

array([[11007,  7073],
       [  477,   526]], dtype=int64)

In [278]:
pred_smote = LR_smote.predict(X_test_scaled)
print("precision: ",precision_score(y_test,pred_smote))
print("recall: ",recall_score(y_test,pred_smote))
print("f1: ",f1_score(y_test,pred_smote))

precision:  0.06921963416238978
recall:  0.5244267198404786
f1:  0.1222971401999535


Checking all the resampling strategies I would go with oversampling as it has the higest precision